In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
import matplotlib as mpl

In [2]:
complete = pd.read_csv('complete_p_logor_se_aic_desc.tsv', sep='\t')
allelotype_map = pd.read_csv('../data/ukb_to_asterisk_names.csv', sep=',')
complete['locus'] = complete['allelotype'].str.split('_').apply(lambda x: x[0])
complete = complete.merge(allelotype_map, left_on='allelotype', right_on='ukb_names')
complete['allelotype'] = complete['literature_names']
complete['phenotype'] = complete['description'].str.split('_').str.join(' ')
complete = complete.drop(columns = ['ukb_names', 'literature_names', 'description'])
complete

,Unnamed: 0,allelotype,phenotype,add_log_OR,add_p,add_se,gen_1_log_OR,gen_1_p,gen_1_se,gen_2_log_OR,gen_2_p,gen_2_se,aic_add,aic_gen,aic_diff,locus
0,0,B*51:01,breast cysts,0.079563,1.000000,0.099909,0.100942,1.0,0.102899,-0.557269,1.0,1.002625,16363.0,16365.0,2.0,B
1,1,B*51:01,bone disorder,0.003354,1.000000,0.101057,0.020568,1.0,0.103931,-0.593249,1.0,1.001456,18592.0,18593.0,1.0,B
2,2,B*51:01,anxiety/panic attacks,-0.074868,1.000000,0.057088,-0.066112,1.0,0.058776,-0.435083,1.0,0.502791,51284.0,51286.0,2.0,B
3,3,B*51:01,connective tissue disorder,0.087449,1.000000,0.074372,0.072625,1.0,0.077842,0.473716,1.0,0.451355,28591.0,28593.0,2.0,B
4,4,B*51:01,svt / supraventricular tachycardia,-0.084748,1.000000,0.078280,-0.112683,1.0,0.081942,0.406370,1.0,0.450813,30474.0,30474.0,0.0,B
5,5,B*51:01,angina,0.032710,1.000000,0.029470,0.013080,1.0,0.030840,0.470100,1.0,0.180300,130589.0,130586.0,-3.0,B
6,6,B*51:01,anal fissure,-0.062281,1.000000,0.076758,-0.039170,1.0,0.078534,-1.199414,1.0,1.001297,31252.0,31252.0,0.0,B
7,7,B*51:01,non-melanoma skin cancer,0.044769,1.000000,0.030289,0.061939,1.0,0.031250,-0.439327,1.0,0.273595,126990.0,126988.0,-2.0,B
8,8,B*51:01,thyroid cancer,0.037863,1.000000,0.167659,0.077255,1.0,0.170989,-11.057364,1.0,191.314664,7505.0,7505.5,0.5,B
9,9,B*51:01,bone metastases / bony secondaries,-0.005800,1.000000,0.092060,-0.001536,1.0,0.095300,-0.126700,1.0,0.709800,21509.0,21511.0,2.0,B


In [3]:
allelotype_counts = complete[complete['add_p'] <= 0.05].groupby('allelotype').count()
allelotype_counts.reset_index(level=0, inplace=True)
allelotype_counts['locus'] = allelotype_counts['allelotype'].str.split('*').apply(lambda x: x[0])
allelotype_counts = allelotype_counts[['allelotype', 'phenotype', 'locus']]
allelotype_counts.columns = ['allelotype', 'num_assoc', 'locus']
allelotype_counts = allelotype_counts.set_index('allelotype')
allelotype_counts = allelotype_counts.sort_values(by='num_assoc')
phenotype_counts = complete[complete['add_p'] <= 0.05].groupby('phenotype').count()['Unnamed: 0']
phenotype_counts = phenotype_counts.sort_values()
locus_counts = complete[complete['add_p'] <= 0.05].groupby('locus').count()['Unnamed: 0']
locus_counts = locus_counts.sort_values()

In [4]:
sns.set_style('whitegrid')
fig = plt.figure(figsize=(60, 20), dpi=450)
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 0.01, 1])

####################################################################################################
#FIRST GRAPH

ax = fig.add_subplot(gs[0, 0])

colors = {'A': 'lightcoral', 
          'B': 'sandybrown', 
          'C': 'blanchedalmond', 
          'DPA1': 'darkgoldenrod', 
          'DPB1': 'olivedrab', 
          'DQA1': 'mediumseagreen', 
          'DQB1': 'turquoise', 
          'DRB1': 'powderblue', 
          'DRB3': 'cornflowerblue', 
          'DRB4': 'mediumpurple', 
          'DRB5': 'orchid'}

ax.barh(allelotype_counts.reset_index()['allelotype'], allelotype_counts['num_assoc'], color=[colors[i] for i in allelotype_counts['locus']], height=1, alpha=0.7)

ax.set_ylabel('Allelotype', size=15)
ax.set_xlabel('Number of Total Significant Phenotype Associations', size=15)
patches, labels = ax.get_legend_handles_labels()

custom_lines = [Line2D([0], [0], color='lightcoral', lw=4, alpha = 0.7),
                Line2D([0], [0], color='sandybrown', lw=4, alpha = 0.7),
                Line2D([0], [0], color='blanchedalmond', lw=4, alpha = 0.7),
                Line2D([0], [0], color='darkgoldenrod', lw=4, alpha = 0.7),
                Line2D([0], [0], color='olivedrab', lw=4, alpha = 0.7),
                Line2D([0], [0], color='mediumseagreen', lw=4, alpha = 0.7),
                Line2D([0], [0], color='turquoise', lw=4, alpha = 0.7),
                Line2D([0], [0], color='powderblue', lw=4, alpha = 0.7),
                Line2D([0], [0], color='cornflowerblue', lw=4, alpha = 0.7),
                Line2D([0], [0], color='mediumpurple', lw=4, alpha = 0.7),
                Line2D([0], [0], color='orchid', lw=4, alpha = 0.7)]
ax.legend(custom_lines, ['A','B','C','DPA1','DPB1','DQA1','DQB1','DRB1','DRB3','DRB4','DRB5'], prop={'size': 20}) 


ax.grid(axis='x', ls='--')
ax.grid(axis='y', b=None)



####################################################################################################
#SECOND GRAPH

ax = fig.add_subplot(gs[0, 1])

ax.barh(locus_counts.keys(), locus_counts, height=1, color=['darkgoldenrod',
                                                  'olivedrab', 
                                                  'orchid',
                                                  'cornflowerblue',
                                                  'lightcoral',
                                                  'mediumpurple',
                                                  'blanchedalmond',
                                                  'mediumseagreen', 
                                                  'sandybrown',
                                                  'turquoise', 
                                                  'powderblue'], alpha=0.7)
ax.set_ylabel('Locus', size=15)
ax.set_xlabel('Number of Total Significant Phenotype Associations', size=15)

ax.grid(axis='x', alpha=0.5, ls='--')
ax.grid(axis='y', b=None)



####################################################################################################
#THIRD GRAPH

ax = fig.add_subplot(gs[0, 3])

ax.barh(phenotype_counts.keys(), phenotype_counts, height=1, color=['powderblue',
                                                'turquoise', 
                                                'sandybrown',
                                                'mediumseagreen', 
                                                'blanchedalmond',
                                                'mediumpurple',
                                                'lightcoral', 
                                                'cornflowerblue',
                                                'orchid',
                                                'olivedrab', 
                                                'darkgoldenrod'], alpha=0.7)
ax.set_ylabel('Phenotype', size=15)
ax.set_xlabel('Number of Total Significant Allelotype Associations', size=15)
ax.grid(axis='x', alpha=0.5, ls='--')
ax.grid(axis='y', b=None)

#####################################################################################################
#TEXT LABELS

t = fig.text(0.11, 0.9, 'a', weight='bold', size=50)
t = fig.text(0.37, 0.9, 'b', weight='bold', size=50)
t = fig.text(0.63, 0.9, 'c', weight='bold', size=50)
plt.savefig("figure.png", dpi=450)